In [1]:
import os
import sys
from tqdm import tqdm
import pandas as pd

# Chemin du dossier src
chemin_src = os.path.join('.', 'src')
sys.path.append(chemin_src)


from functions import Separate_data, clean_table_mp  
from functions import read_and_check_FDN_input_values, read_and_check_USB_input_values
from functions import create_matrix_A_and_C,format_constraints_elements
from functions import format_constraints_qualite, format_constraints_MP
from functions import optimize_with_correction
from functions import solve_linear_program
from functions import  remove_old_recipes
from functions import  gestion_resultats
from functions import  gestion_FDNresultats

In [2]:
def create_optimal_recipe(recette,table_mp, mp_constraints, elmt_quality_constraints,dossier):

    df_mp, df_elmt_and_quality = mp_constraints[recette], elmt_quality_constraints[recette]
    df_contraints_element, df_contraints_qualite, df_mp_dispo, df_mp_indispo = Separate_data(table_mp, df_mp, df_elmt_and_quality)

    # Suppression des matières premières indisponibles dans
    table_mp = clean_table_mp(table_mp, df_mp_indispo)

    # Construction de la matrice A et du vecteur C
    A, C = create_matrix_A_and_C(table_mp, df_mp_dispo)
    
    # Initialisation des listes pour les contraintes
    constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

    # Mettre  les contraintes concernant les éléments
    constraints = format_constraints_elements(df_contraints_element, A,constraints)

    # Mettre  les contraintes concernant la qualité
    constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)

    # Mettre les contraintes concernant les matières premières disponibles
    constraints, bounds = format_constraints_MP(df_mp_dispo, constraints)


    # Résoudre le problème d'optimisation linéaire
    method = 'simplex' #'interior-point' 'simplex'

    # Résolution de la formulation 1
    coefficients = [0.6, 0.65,0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    coefficients = [0.85, 0.9, 0.95,1]
    res1, erreurs1 = optimize_with_correction(table_mp, C, constraints, bounds, method, coefficients)
    
    # Résolution de la formulation 2
    res2, erreurs2 = solve_linear_program(C, constraints, bounds,method)
    
    # # Gestion du resultats USB
    # gestion_resultats(erreurs1, res1, erreurs2, res2, 
    #                   df_mp_dispo, table_mp, constraints, dossier, recette)
    
    # Gestion du resultats pour FDN
    gestion_FDNresultats(erreurs1, res1, erreurs2, res2, df_mp_dispo, table_mp, constraints, dossier, recette)

    return 

In [16]:
dossier_InputsOutputs = os.path.join('.', 'data', 'InputsOutput')
# Suppression du vieux resultats
remove_old_recipes(dossier_InputsOutputs)

# Vérifications des données d'entrée
recette = 'GS' # Nom de la recette
table_mp, mp_constraints, elmt_quality_constraints, errors= read_and_check_FDN_input_values(dossier_InputsOutputs,recette)



df_mp, df_elmt_and_quality = mp_constraints[recette], elmt_quality_constraints[recette]
df_contraints_element, df_contraints_qualite, df_mp_dispo, df_mp_indispo = Separate_data(table_mp, df_mp, df_elmt_and_quality)

# Suppression des matières premières indisponibles dans
table_mp = clean_table_mp(table_mp, df_mp_indispo)

# Construction de la matrice A et du vecteur C
A, C = create_matrix_A_and_C(table_mp, df_mp_dispo)

# Initialisation des listes pour les contraintes
constraints = {'A_eq': {},'b_eq': {},'A_sup': {},'b_sup': {} }

# Mettre  les contraintes concernant les éléments
constraints = format_constraints_elements(df_contraints_element, A,constraints)

# Mettre  les contraintes concernant la qualité
constraints = format_constraints_qualite(df_contraints_qualite, A,constraints)

# Mettre les contraintes concernant les matières premières disponibles
constraints, bounds = format_constraints_MP(df_mp_dispo, constraints)


# Résoudre le problème d'optimisation linéaire
method = 'simplex' #'interior-point' 'simplex'

# Résolution de la formulation 1
coefficients = [0.6, 0.65,0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
coefficients = [0.85, 0.9, 0.95,1]


res1, erreurs1 = optimize_with_correction(table_mp, C, constraints, bounds, method, coefficients)

# Résolution de la formulation 2
res2, erreurs2 = solve_linear_program(C, constraints, bounds,method)




In [17]:
import pandas as pd
import numpy as np
import os
from functions import Impurete_Values, ONO_Values
from functions import THIELMANN_Values, MAYER_Values, Ferrite_Values
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import gc


def construct_FDNresult_dataframe(res, df_mp_dispo, table_mp):
    """
    Construit un DataFrame contenant les résultats de l'optimisation ainsi que les contraintes à respecter.

    Args:
    res: Résultat de l'optimisation.
    df_mp_dispo (DataFrame): DataFrame contenant les données sur les matières premières disponibles.
    table_mp (DataFrame): DataFrame contenant les données sur les matières premières et leurs éléments.
    constraints (dict): Dictionnaire contenant les contraintes du problème.

    Returns:
    DataFrame: DataFrame contenant les résultats de l'optimisation.
    dict: Dictionnaire contenant les contraintes du résultat.
    """
    # Constantes pour les seuils Métalliques
    SEUIL_0 = 1e-20
    SEUIL_1 = 1e-20 # 1e-20 0.01

    # Création du DataFrame df_res
    df_res = df_mp_dispo[['Article', 'Métallique ?']].copy()

    # Ajout de la colonne 'Proportion' avec les valeurs de res.x
    # df_res['Proportion'] = res.x
    df_res['Proportion'] = res.x[:df_mp_dispo[['Prix']].shape[0]]

    # Calcul de la colonne 'Cout' en multipliant 'Proportion' par 'Prix'
    df_res['Cout'] = df_res['Proportion'] * df_mp_dispo['Prix']

    # Filtre des lignes avec des proportions non nulles seulement
    df_res = df_res[df_res['Proportion'] > 0]

    # Tri du DataFrame par 'Proportion' de manière décroissante
    df_res.sort_values(by='Proportion', ascending=False, inplace=True)


    # Filtrage des lignes en fonction de la valeur de la colonne 'Métallique ?' et du seuil correspondant
    df_res = df_res.loc[((df_res['Métallique ?'] == 0) & (df_res['Proportion'] >= SEUIL_0)) |
                        ((df_res['Métallique ?'] == 1) & (df_res['Proportion'] >= SEUIL_1))]

    # Suppression de la colonne 'Métallique ?' qui n'est plus nécessaire
    df_res.drop(columns=['Métallique ?'], inplace=True)


    # Pour le deuxieme fichier excel
    # Calcul des proportions des éléments dans la fonte
    cols_elements = table_mp.columns[2:].tolist()
    proportions_elements = table_mp[cols_elements].mul(df_res['Proportion'], axis=0).sum()

    # creer un nouveau dataframme df_resElements
    df_resElements = pd.DataFrame([proportions_elements], columns=cols_elements)


    # Calcul des valeurs des indicateurs qualité
    impurete_values, ono_values = np.array(list(Impurete_Values.values())), np.array(list(ONO_Values.values()))
    thielmann_values, mayer_values = np.array(list(THIELMANN_Values.values())), np.array(list(MAYER_Values.values()))
    ferrite_values = np.array(list(Ferrite_Values.values()))
    df_resElements['Impurete'] = impurete_values @ proportions_elements
    df_resElements['ONO'] = ono_values @ proportions_elements
    df_resElements['THIELMANN'] = thielmann_values @ proportions_elements
    df_resElements['MAYER'] = mayer_values @ proportions_elements
    df_resElements['Ferrite'] = 92.3 + ferrite_values @ proportions_elements


    # Cette partie est remplie uniquement par Salma
    df_resElements['Rm'] = np.nan
    df_resElements['Moyenne allongement'] = np.nan


    return df_res, df_resElements



In [5]:
Impurete_Values

{'C': 0,
 'Si': 0,
 'Mn': 0.44,
 'Cu': 4.9,
 'Cr': 0.37,
 'P': 5.6,
 'Ni': 0.37,
 'Mo': 7.9,
 'Sn': 39.0,
 'Sb': 0,
 'Al': 0,
 'S': 0,
 'Mg': 0,
 'Pb': 0,
 'Ti': 4.4,
 'As': 0,
 'Bi': 0,
 'V': 0}

In [19]:
# Constantes pour les seuils Métalliques
SEUIL_0 = 1e-20
SEUIL_1 = 1e-20 # 1e-20 0.01

# Création du DataFrame df_res
df_res = df_mp_dispo[['Article', 'Métallique ?']].copy()

# Ajout de la colonne 'Proportion' avec les valeurs de res.x
# df_res['Proportion'] = res.x
df_res['Proportion'] = res1.x[:df_mp_dispo[['Prix']].shape[0]]

# Calcul de la colonne 'Cout' en multipliant 'Proportion' par 'Prix'
df_res['Cout'] = df_res['Proportion'] * df_mp_dispo['Prix']

# Filtre des lignes avec des proportions non nulles seulement
df_res = df_res[df_res['Proportion'] > 0]

# Tri du DataFrame par 'Proportion' de manière décroissante
df_res.sort_values(by='Proportion', ascending=False, inplace=True)


# Filtrage des lignes en fonction de la valeur de la colonne 'Métallique ?' et du seuil correspondant
df_res = df_res.loc[((df_res['Métallique ?'] == 0) & (df_res['Proportion'] >= SEUIL_0)) |
                    ((df_res['Métallique ?'] == 1) & (df_res['Proportion'] >= SEUIL_1))]

# Suppression de la colonne 'Métallique ?' qui n'est plus nécessaire
df_res.drop(columns=['Métallique ?'], inplace=True)


# Pour le deuxieme fichier excel
# Calcul des proportions des éléments dans la fonte
# Sélectionner des colonnes spécifiques après la fusion
cols_elements = table_mp.columns[2:].tolist()
df_commun = pd.merge(df_res[['Article', 'Proportion']], table_mp[cols_elements + ['Article']], on='Article')

# Calcul des proportions des éléments après la fusion
proportions_elements = df_commun[cols_elements].mul(df_commun['Proportion'], axis=0).sum()


# creer un nouveau dataframme df_resElements
df_resElements = pd.DataFrame([proportions_elements], columns=cols_elements)


# Calcul des valeurs des indicateurs qualité
impurete_values, ono_values = np.array(list(Impurete_Values.values())), np.array(list(ONO_Values.values()))
thielmann_values, mayer_values = np.array(list(THIELMANN_Values.values())), np.array(list(MAYER_Values.values()))
ferrite_values = np.array(list(Ferrite_Values.values()))
df_resElements['ONO'] = ono_values @ proportions_elements
df_resElements['Impurete'] = impurete_values @ proportions_elements
df_resElements['THIELMANN'] = thielmann_values @ proportions_elements
df_resElements['MAYER'] = mayer_values @ proportions_elements
df_resElements['Ferrite'] = 92.3 + ferrite_values @ proportions_elements


# Cette partie est remplie uniquement par Salma
df_resElements['Rm'] = np.nan
df_resElements['Moyenne allongement'] = np.nan

df_res["Proportion"].sum()

0.6706913021618903

In [20]:
df_res

,Article,Proportion,Cout
4,Retour Bleu,0.500000,290.000000
3,FRITE HAUT SILICIUM,0.147059,66.529412
2,Carbone,0.019108,26.769736
1,Disque,0.004525,2.624434


In [15]:
df_resElements

,C,Si,Mn,Cu,Cr,P,Ni,Mo,Sn,Sb,...,As,Bi,V,ONO,Impurete,THIELMANN,MAYER,Ferrite,Rm,Moyenne allongement
0,3.6,1.8,0.15,0.031787,0.025452,0.020452,0.01,0.0,0.005,0.0,...,0.0,0.0,0.0,0.07224,0.544409,0.0115,0.0,71.162873,NaN,NaN


##### FDN

In [13]:
if __name__ == "__main__":
    dossier_InputsOutputs = os.path.join('.', 'data', 'InputsOutput')
    # Suppression du vieux resultats
    remove_old_recipes(dossier_InputsOutputs)

    # Vérifications des données d'entrée
    recette = 'GS' # Nom de la recette
    table_mp, mp_constraints, elmt_quality_constraints, errors= read_and_check_FDN_input_values(dossier_InputsOutputs,recette)

    # Resolutions du nouveau probleme 1 et 2
    if not errors :
        create_optimal_recipe(recette, table_mp, mp_constraints,elmt_quality_constraints,dossier_InputsOutputs)

        

Les erreurs des deux formulations pour la recette GS ont été enregistrées dans un fichier.


##### USB

In [3]:
if __name__ == "__main__":
    chemin_fichier = os.path.join('.', 'data', 'recipe_optimization_data.xlsm')
    # On recupere le chemin du dossier data
    dossier_data = os.path.dirname(chemin_fichier)
    # Suppression du vieux resultats
    remove_old_recipes(dossier_data)

    # Vérifications des données d'entrée
    Recettes = [col for col in pd.read_excel(chemin_fichier, engine='openpyxl', sheet_name=2).columns if 'Unnamed' not in col] # 'calamine'
    table_mp, mp_constraints, elmt_quality_constraints, errors= read_and_check_USB_input_values(chemin_fichier)

    # Resolutions du nouveau probleme
    if not errors :
        for recette in tqdm(Recettes, desc="Processing recipes", unit="recipe"):
            create_optimal_recipe(recette, table_mp, mp_constraints,elmt_quality_constraints,dossier_data)



Processing recipes:  40%|████      | 2/5 [00:00<00:00,  6.34recipe/s]

Le problème pour la recette GS 400-15 admet une solution pour les deux formulations.
Le problème pour la recette GS 450-10 admet une solution pour les deux formulations.


Processing recipes:  80%|████████  | 4/5 [00:00<00:00,  6.10recipe/s]

Le problème pour la recette GS 500-7 admet une solution pour les deux formulations.
Le problème pour la recette GS 600-3 admet une solution pour les deux formulations.


Processing recipes: 100%|██████████| 5/5 [00:00<00:00,  5.55recipe/s]

Le problème pour la recette GL 250 admet une solution pour les deux formulations.
